In [1]:
import pandas as pd
import numpy as np
import yaml
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error
from C_functions_opti import get_LHS_samples, model_optimization, plot_estimation

##### 1. Load all data that is needed

In [2]:
# Load experimental data
df_exp = pd.read_csv('data/data_combined.csv')
biomass_exp = df_exp['Biomass [g/L]']
substrate_exp = df_exp['Glucose [g/L]']

In [3]:
# Load parameters from YAML file
with open('config/parameters.yml', 'r') as file:
    param = yaml.safe_load(file)

mu_max = param['mu_max']
X_max = param['X_max']
Ks = param['Ks']
Ks_qs = param['Ks_qs']
Ki = param['Ki']
Yxs = param['Yxs']
qs_max = param['qs_max']
m_s = param['m_s']
lag = param['lag']

##### 2. Define all possible equations for mu and qs

In [4]:
# -- MONOD / insert: mu_max, c_glucose, Ks
mu0 = lambda mu_max, c_glucose, Ks: mu_max * c_glucose / (c_glucose + Ks)
# -- LOGISTIC / insert: mu_max, c_biomass, X_max
mu1 = lambda mu_max, c_biomass, X_max: mu_max * (1 - (c_biomass/ X_max)) 
# -- MONOD + LOGISTIC / insert: mu_max, c_glucose, Ks, c_biomass, X_max
mu2 = lambda mu_max, c_glucose, Ks, c_biomass, X_max: mu_max * (c_glucose / (c_glucose + Ks)) * (1 - (c_biomass/ X_max))
# -- MONOD + LOGISTIC + INHIBITION / insert: mu_max, c_glucose, Ks, Ki, c_biomass, X_max
mu3 = lambda mu_max, c_glucose, Ks, Ki, c_biomass, X_max: mu_max * (c_glucose / (c_glucose + Ks + (c_glucose**2/ Ki))) * (1 - (c_biomass/ X_max))

In [5]:
# -- MONOD / insert: qs_max, c_glucose, Ks_qs
qs0 = lambda qs_max, c_glucose, Ks_qs: qs_max * c_glucose / (Ks_qs + c_glucose)
# -- MONOD + NON COMPETITIVE INHIBITION / insert: qs_max, c_glucose, Ks_qs, Ki, glu_met
qs1 = lambda qs_max, c_glucose, Ks_qs, Ki, glu_met: qs_max * c_glucose / (Ks_qs + c_glucose) * (Ki / (Ki + glu_met))
# -- YIELD / insert: Yxs, f_glucose, V
qs2 = lambda Yxs, f_glucose, V: 1/Yxs * f_glucose / V #NOT SURE IF CORRECT because g_S/g_X but not per h
# -- MONOD + METABOLIZED GLU / insert: qs_max, c_glucose, Ks_qs, glu_met, lag
qs3 = lambda qs_max, c_glucose, Ks_qs, glu_met, lag: qs_max * c_glucose / (Ks_qs + c_glucose) * (1 / (np.exp(glu_met * lag)))

In [6]:
mu_all=[mu0, mu1, mu2, mu3]
qs_all=[qs0, qs1, qs2, qs3]

##### 3. Define functions that are needed

In [7]:
# Root mean squared error is the objective function
def objective_function(parameters, mu_eq, num_mu, qs_eq, num_qs):
    # Solve the model using the optimal parameters
    time_pred, biomass_pred, substrate_pred, volume_pred = model_optimization(param, parameters, mu_eq, num_mu, qs_eq, num_qs)  # Solve the model using the current parameters
    biomass = pd.concat([biomass_exp, pd.Series(biomass_pred)], axis=1, keys=['biomass_exp', 'biomass_pred']).dropna()
    biomass_exp_ = biomass['biomass_exp'].values
    biomass_pred_ = biomass['biomass_pred'].values
    mse_x = mean_squared_error(biomass_exp_, biomass_pred_)  # Calculate mean squared error for biomass

    glucose = pd.concat([substrate_exp, pd.Series(substrate_pred)], axis=1, keys=['substrate_exp', 'substrate_pred']).dropna()
    substrate_exp_ = glucose['substrate_exp'].values
    substrate_pred_ = glucose['substrate_pred'].values
    mse_s = mean_squared_error(substrate_exp_, substrate_pred_)  # Calculate mean squared error for substrate
    
    # Calculate the combined rmse
    mse = (mse_x + mse_s)/2
    rmse = np.sqrt(mse)  # Calculate root mean squared error
    return rmse, time_pred, biomass_pred, substrate_pred, volume_pred

##### 4. Define set of parameters in config file

In [8]:
# Extract the parameter combination and the number of the set in order to save it in the correct folder
est_mu_max = param['est_mu_max'] # ['set0']
est_Ks = param['est_ks']

In [9]:
# Set the number of samples and parameters
num_samples = 100
num_parameters = 9

# Define the ranges for each parameter
parameter_bounds = [
    [0.01, 0.6],    # Range for parameter 1 mu_max
    [18, 25],       # Range for parameter 2 X_max
    [0.1, 10.0],    # Range for parameter 3 - Ks
    [0.01, 0.1],    # Range for parameter 4 - Ks_qs
    [6.0, 8.0],     # Range for parameter 5 - Ki
    [0.4, 0.5],     # Range for parameter 6 - Yxs
    [0.5, 1.5],     # Range for parameter 7 - qs_max
    [0.05, 0.15],   # Range for parameter 8 - m_s
    [0.001, 0.02],    # Range for parameter 9 - lag
]

In [10]:
LHS_samples = get_LHS_samples(num_samples, num_parameters, parameter_bounds)
LHS_samples.shape

(100, 9)

##### 5. Run it

In [11]:
df_all_sets = pd.DataFrame(columns=['set', 'mu', 'qs', 'mu_max', 'X_max', 'Ks', 'Ks_qs', 'Ki', 'Yxs', 'qs_max', 'm_s', 'lag', 'rmse'])
for set_num in range(LHS_samples.shape[0]):
    # Save all parameters and equations and the RMSE in a dataframe
    ## with the beginning of one set a new rmse_overview will be created
    rmse_one_set = []
    #key = f'set{set_num}' ; init_p = est_mu_max[key]
    init_p = list(LHS_samples[set_num, :])
    for i in range(len(mu_all)):
        for j in range(len(qs_all)):
            # Define the combination of equations
            mu_eq = mu_all[i]; num_mu = i
            qs_eq = qs_all[j]; num_qs = j

            # Make the predictions and calculate the error
            rmse, time_pred, biomass_pred, substrate_pred, volume_pred = objective_function(init_p, mu_eq, num_mu, qs_eq, num_qs)
            # save the parameters in a dataframe
            append_list=[set_num, i, j, init_p[0], init_p[1], init_p[2], init_p[3], init_p[4], init_p[5], init_p[6], init_p[7], init_p[8], round(rmse, 3)]
            rmse_one_set.append(append_list)

            # Make a plot and save it
            title = f'set{set_num}/ mu{i} - qs{j} - rmse: {round(rmse, 3)}'
            plot_name = f'set{set_num}_mu{i}_qs{j}_rmse{int(rmse)}'

            if rmse <= float(10):
                plot_estimation(time_pred, biomass_pred, substrate_pred, volume_pred, title, plot_name, set_num)

    # save the parameters of one set
    df_1set = pd.DataFrame(rmse_one_set, columns=['set', 'mu', 'qs', 'mu_max', 'X_max', 'Ks', 'Ks_qs', 'Ki', 'Yxs', 'qs_max', 'm_s', 'lag', 'rmse'])
    df_all_sets = pd.concat([df_all_sets,df_1set], axis=0, ignore_index=True)


df_all_sets.sort_values(by=['rmse'], ascending=True, inplace=True)
df_all_sets.to_csv(f'data/estimation/LHS_sampling/data.csv')

df_all_sets.head(10)

C:\Users\dxp2904\AppData\Local\Temp\ipykernel_14092\465046859.py:8: RuntimeWarning: overflow encountered in exp
  qs3 = lambda qs_max, c_glucose, Ks_qs, glu_met, lag: qs_max * c_glucose / (Ks_qs + c_glucose) * (1 / (np.exp(glu_met * lag)))
C:\Users\dxp2904\AppData\Local\Temp\ipykernel_14092\465046859.py:8: RuntimeWarning: overflow encountered in exp
  qs3 = lambda qs_max, c_glucose, Ks_qs, glu_met, lag: qs_max * c_glucose / (Ks_qs + c_glucose) * (1 / (np.exp(glu_met * lag)))
C:\Users\dxp2904\AppData\Local\Temp\ipykernel_14092\465046859.py:8: RuntimeWarning: overflow encountered in exp
  qs3 = lambda qs_max, c_glucose, Ks_qs, glu_met, lag: qs_max * c_glucose / (Ks_qs + c_glucose) * (1 / (np.exp(glu_met * lag)))
C:\Users\dxp2904\AppData\Local\Temp\ipykernel_14092\465046859.py:8: RuntimeWarning: overflow encountered in exp
  qs3 = lambda qs_max, c_glucose, Ks_qs, glu_met, lag: qs_max * c_glucose / (Ks_qs + c_glucose) * (1 / (np.exp(glu_met * lag)))
C:\Users\dxp2904\AppData\Local\Temp\ipyk

,set,mu,qs,mu_max,X_max,Ks,Ks_qs,Ki,Yxs,qs_max,m_s,lag,rmse
156,9,3,0,0.550741,20.487435,2.580617,0.093960,7.311930,0.409555,1.358982,0.148821,0.005412,2.966
1543,96,1,3,0.158542,19.808707,2.755936,0.062110,6.698058,0.417436,0.998364,0.073734,0.007030,3.567
743,46,1,3,0.145007,24.999547,2.817220,0.035125,7.391400,0.413295,1.138623,0.082984,0.011765,3.655
1540,96,1,0,0.158542,19.808707,2.755936,0.062110,6.698058,0.417436,0.998364,0.073734,0.007030,3.916
724,45,1,0,0.207232,19.301242,3.845603,0.064616,7.017024,0.493312,0.647082,0.084262,0.015402,3.977
159,9,3,3,0.550741,20.487435,2.580617,0.093960,7.311930,0.409555,1.358982,0.148821,0.005412,4.037
615,38,1,3,0.135897,20.379806,4.334269,0.052367,6.550460,0.464337,1.374369,0.112274,0.019020,4.872
500,31,1,0,0.124975,22.351318,3.942122,0.056132,7.898782,0.439156,1.319914,0.065199,0.001816,4.873
1045,65,1,1,0.225940,18.798505,8.298892,0.016141,6.632580,0.450680,1.473834,0.131891,0.009873,5.222
503,31,1,3,0.124975,22.351318,3.942122,0.056132,7.898782,0.439156,1.319914,0.065199,0.001816,5.314


In [12]:
df_LHS = pd.read_csv('data/estimation/LHS_sampling/data.csv')